In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Making a simple CNN model to train and make predictions on the MINST digits dataset.Upvote if u like 
Importing Libraries

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import LearningRateScheduler

Importing Dataset

In [ ]:
train = pd.read_csv('../input/digit-recognizer/train.csv')
test = pd.read_csv('../input/digit-recognizer/test.csv')

In [ ]:
train.head(10)

In [ ]:
test.head(10)

As seen above, these are datasets of pixels by image, with each row representing a seperate image of 784 pixels (likely 28 high X 28 wide). The training set contains 42000 images, while the testing set contains 28000. This appears to be monohromatic, as there is only one layer. Were they not black and white, we could expect three different sets of values for each pixel, representing RGB

In [ ]:
# Checking missing values
sns.heatmap(train.isnull())


Checking the null value in the dataset.Using seaborn to plot if there is any null values 

In [ ]:
sns.heatmap(test.isnull())

Checking that the dataset we have is balanced or not.Since the data is well balanced upon the 10 categories given to us so we don't need to upsample or downsample the classes.

In [ ]:
X_train=train.iloc[:,1:]
Y_train=train.iloc[:,0]
g=sns.countplot(Y_train)


Pretty standard configuration: 0 is black, 255 is white. We should adjust the values so they fall between 0 and 1, which makes it easier for the model to handle later. We should also reshape and re-type the data to the required format. Let's wrap it all up in a function.
So we normailize the data so that we get values between 0 and 1 so model can converge faster.

In [ ]:
X_train=X_train/255.0
test=test/255.0

Train and test images (28px x 28px) has been stock into pandas.Dataframe as 1D vectors of 784 values. We reshape all data to 28x28x1 3D matrices.

Keras requires an extra dimension in the end which correspond to channels. MNIST images are gray scaled so it use only one channel. For RGB images, there is 3 channels, we would have reshaped 784px vectors to 28x28x3 3D matrices.

In [ ]:

X_train = X_train.values.reshape(-1, 28, 28, 1)
test = test.values.reshape(-1, 28, 28, 1)

In [ ]:
X_train.shape

Spliting the data into train and validation.

In [ ]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(X_train,Y_train,test_size=0.2)

These images are very small. Also, if you go look at images of what these numbers look like (sorry for not including examples in here), you'll see that all of the numbers are fairly standard: right side up, not mirrored, not too tilted, etc. Thus, we shouldn't need to do too much to avoid overfitting; we'll just shift things around, rotate slightly, and zoom a bit.
Now we do data augumentation.It is a technique to increase the size of the dataset by applying various rotations and various other factors so that model can train on more images to get high accuracy.

In [ ]:
data_gen = ImageDataGenerator(
            rotation_range=12,
            width_shift_range=0.12,
            height_shift_range=0.12,
            shear_range=0.12,
            validation_split=0.2,)

In [ ]:
data_gen.fit(x_train)
data_gen.fit(x_test)

Encoding our outing into 10 differents categories

In [ ]:

y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

The first is the convolutional (Conv2D) layer. It is like a set of learnable filters. I choosed to set 16 filters for the firsts conv2D layer and 32 filters for the two last ones. Each filter transforms a part of the image (defined by the kernel size) using the kernel filter. The kernel filter matrix is applied on the whole image. Filters can be seen as a transformation of the image.

The CNN can isolate features that are useful everywhere from these transformed images (feature maps).

The second important layer in CNN is the pooling (MaxPool2D) layer. This layer simply acts as a downsampling filter. It looks at the 2 neighboring pixels and picks the maximal value. These are used to reduce computational cost, and to some extent also reduce overfitting. We have to choose the pooling size (i.e the area size pooled each time) more the pooling dimension is high, more the downsampling is important.

Combining convolutional and pooling layers, CNN are able to combine local features and learn more global features of the image.

Dropout is a regularization method, where a proportion of nodes in the layer are randomly ignored (setting their wieghts to zero) for each training sample. This drops randomly a propotion of the network and forces the network to learn features in a distributed way. This technique also improves generalization and reduces the overfitting.

'relu' is the rectifier (activation function max(0,x). The rectifier activation function is used to add non linearity to the network.

The Flatten layer is use to convert the final feature maps into a one single 1D vector. This flattening step is needed so that you can make use of fully connected layers after some convolutional/maxpool layers. It combines all the found local features of the previous convolutional layers.

In the end i used the features in two fully-connected (Dense) layers which is just artificial an neural networks (ANN) classifier. In the last layer(Dense(10,activation="softmax")) the net outputs distribution of probability of each class.

In [ ]:
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization
model = Sequential()
model.add(Conv2D(filters = 64, kernel_size = (5,5),padding = 'Same', activation ='relu', input_shape = (28,28,1)))
model.add(BatchNormalization())

model.add(Conv2D(filters = 64, kernel_size = (5,5),padding = 'Same', activation ='relu'))
model.add(BatchNormalization())

model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(BatchNormalization())

model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = 'Same',  activation ='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Dense(10, activation = "softmax"))
model.summary()

In [ ]:
model.summary()

We now have a model framework with about 1022858 parameters. Next, we need to compile it. We'll try using the 'Adam' optomizer function, which will adapt the learning rate, along with categorical crossentropy for loss (becuase we are looking for categorical classification), and measure accuracy.

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

All that's left is to train it. We'll choose 15 epochs and a batch size of 64. The images are pretty small, so this shouldn't take too long.I have run this model for 15 epochs you can run for 30 epoch for best result.

In [ ]:
!pip install visualkeras
import visualkeras

visualkeras.layered_view(model)

In [ ]:
from keras.callbacks import EarlyStopping, Callback
class myCallback(Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>0.999):
            print("\nReached 99.9% accuracy so cancelling training!")
            self.model.stop_training = True

# Instantiate callback
mycallback = myCallback()


early_stopping_callback = EarlyStopping(monitor='val_accuracy', 
                                        patience=3,
                                        verbose = 2,
                                        restore_best_weights=True)


In [ ]:
# defining model parameters 
batch_size = 1000
epochs = 50
# Fit the Model
history = model.fit_generator(data_gen.flow(x_train, y_train, batch_size = batch_size), epochs = epochs, 
                              validation_data = (x_test, y_test), verbose=1, 
                              steps_per_epoch=x_train.shape[0] // batch_size,
                              callbacks = [mycallback,early_stopping_callback]
                             )

 

Checking how good my model is working on loss.

In [ ]:
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.title("Model Loss")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend();

In [ ]:
pred = np.argmax(model.predict(test), axis=1)
    
sub_df = {'ImageId':list(range(1, len(test) + 1)),'Label':pred}
submission = pd.DataFrame(sub_df).astype('int')
submission.head()

In [ ]:
submission.to_csv('submission.csv', index=False)